# Instructions:
## Provide the path to the 3 csv's in variables cell.
## Run all the cells as it is.
## The result will be saved at {result_path}.csv


# File Generation

# Variables

In [ ]:
#Path to the folder containing evaluation dataset(4 csv's)
# test_data_path='/content/Final_data' 

#Path to store result file
result_path='/content/result.csv'

#Name to store themewise file and delade predicted para file
m='pred_para'

#Provide paths to the evaluation files
theme_path = 'https://drive.google.com/uc?export=download&id=1H7mElIl0WO1GNwhbjHF5KCMDk2n685qi'
ques_data_path = 'https://drive.google.com/uc?export=download&id=1H0DAi346uAS877bEP2akDFSytxX4pRVY'
para_data_path ='https://drive.google.com/uc?export=download&id=1GNHhH81J1pEZSB-OSGRTtqIUQCCNhJ6I'


# Imports

In [ ]:
!sudo apt install megatools

In [ ]:
#Variables not to be changed
seed = 42
threshold=0.1
similarity_method='dot'
synthetic_data=False
squad_dev_data=False

In [ ]:
%cd /content

In [ ]:
!git clone https://github.com/castorini/dhr.git

In [ ]:
%cd dhr
!git clone https://huggingface.co/jacklin/DeLADE-CLS-P

In [ ]:
!megadl --path /content/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!luAz2IgI!q0R7urmIfhiTLhRpiBqs18Vpuriw6fVnQk_2ZkBPiBU'
!megadl --path /content/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!YmYBQZha!TjnxWDN0V_vp7-c3pxPLPmFPWW60UGy-yPuaaxrheD4'
!megadl --path /content/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!wz4VQYqR!oYV5DTqXhJ9BFwTeJE_BbeTnTlWQ8_JijUhgUhse9WE'
!megadl --path /content/dhr/tevatron/datasets/beir/ 'https://mega.nz/#!8uIBhbgS!RUmCXrIYNfFeUahldqQ6jL2duHuTozJA6q_HpWi_SBU'

In [ ]:
# Allowed to make changes.
!pip install sentence-transformers
!pip install beir
!pip install datasets
!sudo apt install megatools

from sklearn.metrics.pairwise import cosine_similarity
import collections
import gdown
import json
import pandas as pd
import re
import string
import timeit
from ast import literal_eval
import pandas as pd
from google.colab import drive
from sentence_transformers import SentenceTransformer,util
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline
import time
import torch.nn.utils.prune as prune
import numpy as np
import os
from transformers import pipeline,set_seed
import torch
torch.manual_seed(42)
set_seed(42)


# Conversion to beir format

The evaluation data will be converted into beir format data.

In [ ]:
#Change the path to store beir format of evaluation data,if required
beir_data_path='/content/beir_format' 

In [ ]:
import os
os.mkdir(beir_data_path)

In [ ]:
sample_ground_truth_path = 'https://drive.google.com/uc?export=download&id=1--M9XlXEzOLDPCDT69LjmAsQYRPA2gZ2S'
para_df=pd.read_csv(para_data_path)
ques_df=pd.read_csv(ques_data_path)
theme_df=pd.read_csv(theme_path)
ground_df=pd.read_csv(sample_ground_truth_path)

In [ ]:
# ground_df.question_id = ques_df.id

In [ ]:
para_df.rename(columns={'theme':'title'},inplace=True)
para_df.rename(columns={'paragraph':'text'},inplace=True)
para_df.rename(columns={'id':'_id'},inplace=True)
para_df['_id'] = para_df['_id'].apply(lambda x: str(x))
para_df.to_csv(f'{beir_data_path}/corpus.csv',index=False)

In [ ]:
ques_df

In [ ]:
ques_df.rename(columns={'theme':'title'},inplace=True)
ques_df.rename(columns={'question':'text'},inplace=True)
ques_df.rename(columns={'id':'_id'},inplace=True)
ques_df['_id'] = ques_df['_id'].apply(lambda x: str(x))

In [ ]:
ground_df.pop('answers')

In [ ]:
# ground_df=ground_df.iloc[:,[1,2]]
ground_df['question_id'] = ground_df['question_id'].apply(lambda x: str(x))
ground_df['paragraph_id'] = ground_df['paragraph_id'].apply(lambda x: str(x))
ground_df.rename(columns={'paragraph_id':'corpus-id'},inplace=True)
ground_df.rename(columns={'question_id':'query-id'},inplace=True)
ground_df['score']=1

In [ ]:
import json
import os
themes=ques_df['title'].unique()
for theme in themes:
  os.mkdir(f'{beir_data_path}/{theme}')
  os.mkdir(f'{beir_data_path}/{theme}/qrels')
  corpus_df = para_df[para_df["title"]==theme]
  corpus_df.to_json(f'{beir_data_path}/{theme}/corpus.jsonl',orient='records',lines=True)
  queries_df=ques_df[ques_df["title"]==theme]
  queries_df.to_json(f'{beir_data_path}/{theme}/queries.jsonl',orient='records',lines=True)
  lst=queries_df['_id'].tolist()
  # print(theme)
  start=int(lst[0])-1
  end=int(lst[len(queries_df)-1])
  qrels=ground_df[start:end]
  start = theme_df[theme_df['theme']==theme]['start'].tolist()[0]
  end = theme_df[theme_df['theme']==theme]['end'].tolist()[0]
  qrels=ground_df.iloc[start-1:end]
  qrels.to_csv(f'{beir_data_path}/{theme}/qrels/test.tsv',index=False,sep='\t')

# Conversion to beir format (For Questions on DevRev Theme)

The evaluation data will be converted into beir format data.

In [ ]:
#Change the path to store beir format of evaluation data,if required
beir_data_path='/content/beir_format' 

In [ ]:
import os
os.mkdir(beir_data_path)

In [ ]:
sample_ground_truth_path = 'https://drive.google.com/uc?export=download&id=1--M9XlXEzOLDPCDT69LjmAsQYRPA2gZ2'
para_df=pd.read_csv(para_data_path)
ques_df=pd.read_csv(ques_data_path)
theme_df=pd.read_csv(theme_path)
ground_df=pd.read_csv(sample_ground_truth_path)

In [ ]:
para_df=para_df.loc[para_df['theme']=='DevRev']
ques_df=ques_df.loc[ques_df['theme']=='DevRev']
theme_df=theme_df.loc[theme_df['theme']=='DevRev']
start = theme_df[theme_df['theme']=='DevRev']['start'].tolist()[0]
end = theme_df[theme_df['theme']=='DevRev']['end'].tolist()[0]
ground_df=ground_df.iloc[start-1:end]

In [ ]:
para_df.rename(columns={'theme':'title'},inplace=True)
para_df.rename(columns={'paragraph':'text'},inplace=True)
para_df.rename(columns={'id':'_id'},inplace=True)
para_df['_id'] = para_df['_id'].apply(lambda x: str(x))
para_df.to_csv(f'{beir_data_path}/corpus.csv',index=False)

In [ ]:
ques_df.rename(columns={'theme':'title'},inplace=True)
ques_df.rename(columns={'question':'text'},inplace=True)
ques_df.rename(columns={'id':'_id'},inplace=True)
ques_df['_id'] = ques_df['_id'].apply(lambda x: str(x))

In [ ]:
ground_df.pop('answers')

In [ ]:
# ground_df=ground_df.iloc[:,[1,2]]
ground_df['question_id'] = ground_df['question_id'].apply(lambda x: str(x))
ground_df['paragraph_id'] = ground_df['paragraph_id'].apply(lambda x: str(x))
ground_df.rename(columns={'paragraph_id':'corpus-id'},inplace=True)
ground_df.rename(columns={'question_id':'query-id'},inplace=True)
ground_df['score']=1

In [ ]:
import json
import os
themes=ques_df['title'].unique()
for theme in themes:
  os.mkdir(f'{beir_data_path}/{theme}')
  os.mkdir(f'{beir_data_path}/{theme}/qrels')
  corpus_df = para_df[para_df["title"]==theme]
  corpus_df.to_json(f'{beir_data_path}/{theme}/corpus.jsonl',orient='records',lines=True)
  queries_df=ques_df[ques_df["title"]==theme]
  queries_df.to_json(f'{beir_data_path}/{theme}/queries.jsonl',orient='records',lines=True)
  lst=queries_df['_id'].tolist()
  # print(theme)
  start=int(lst[0])-1
  end=int(lst[len(queries_df)-1])
  qrels=ground_df[start:end]
  start = theme_df[theme_df['theme']==theme]['start'].tolist()[0]
  end = theme_df[theme_df['theme']==theme]['end'].tolist()[0]
  qrels=ground_df
  qrels.to_csv(f'{beir_data_path}/{theme}/qrels/test.tsv',index=False,sep='\t')

# Delade generate embeddings


Run this cell to generate embeddings.
Embeddings will be saved at `/content/new_delade_embeddings_corpus.npy`

In [ ]:
%cd '/content/dhr'

In [ ]:
emb_delade_dhr_path='/content/dhr'

In [ ]:
!pip install torch>=1.7.0
# !pip install transformers==4.15.0
!pip install pyserini
!pip install beir
!pip install datasets

In [ ]:
import pandas as pd
para_df=pd.read_csv(f'{beir_data_path}/corpus.csv')

In [ ]:
#Will be generated in /content/new_delade_embeddings_corpus.npy
# Ignore the error. Some changes have been made for reducing the embedding generation time
themes=para_df['title'].unique()
for theme in themes:
  !python -m tevatron.datasets.beir.encode_and_retrieval_emb --theme f'{theme}' --model_name_or_path '/content/dhr/DeLADE-CLS-P' --dataPath f'{beir_data_path}'

In [ ]:
embeddings_path='/content/new_delade_embeddings_corpus.npy'

# Delade load embeddings

In [ ]:
import numpy as np
ab=np.load(embeddings_path,allow_pickle=True)
all_theme_embeddings=ab.item()

In [ ]:
!pip install --upgrade transformers

# Load QA base 

In [ ]:
%cd /content

In [ ]:
nlp=None

In [ ]:
model_name = "mrm8488/electra-small-finetuned-squadv2"
# qa_model_path='/content/drive/MyDrive/Assets/Model Checkpoints/electra-small-base.pt'
# model = torch.load(qa_model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# nlp = pipeline("question-answering", model = model,tokenizer=tokenizer)

In [ ]:
nlp = pipeline("question-answering", model = model_name)

# Load Cross Encoder

In [ ]:
%cd /content

In [ ]:
!megadl --path /content/ 'https://mega.nz/#!8zoUjZKL!iSOdg7ySmNut-mUaRDI0nO5uMZRJLg9R59DxLP3aQvU'

In [ ]:
!unzip /content/minilml4-v2_17-20230205T094107Z-001.zip

In [ ]:
from sentence_transformers import CrossEncoder
cross_enc_ft_model='/content/minilml4-v2_17'
cross_model = CrossEncoder(cross_enc_ft_model,max_length=512)


# Evaluation at k=3


In [ ]:
 %cd /content/dhr

In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  global_model = nlp
  return global_model
def Average(lis):
  return sum(lis)/len(lis)

In [ ]:

para_df2=pd.read_csv(f'{beir_data_path}/corpus.csv')
para_df=pd.read_csv(para_data_path)

In [ ]:
# Allowed to make changes.
#for direct delade(k=5)
def pred_theme_ans(questions, theme_model, pred_out):
  theme = questions[0]['theme']
  if(synthetic_data==True):
    !python -m tevatron.datasets.beir.encode_and_retrieval_syn --theme f'{theme}' --model_name_or_path /content/drive/MyDrive/Assets/delade/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 
  elif(squad_dev_data==True):
    !python -m tevatron.datasets.beir.encode_and_retrieval_squad --theme f'{theme}' --model_name_or_path /content/drive/MyDrive/Assets/delade/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 
  else:
    !python -m tevatron.datasets.beir.encode_and_retrieval_new --theme f'{theme}' --model_name_or_path /content/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 

  for question in questions:
    ans = {}
    ans['question_id'] = question['id']    
    res5=pd.read_csv(f'/content/{m}_res.csv')
    l=len(para_df2.loc[para_df2['title']==theme].text.unique())
    
    if(l>=3):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].C.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]    
    elif(l==2):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]   
    elif(l==1):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]
    scores = cross_model.predict([(question['question'],p) for p in pred_para])
    best = np.argmax(scores)
    result = theme_model(question=question['question'], context=pred_para[best])
    answer = result['answer']
    if result['score'] < threshold:
      ans['paragraph_id'] = -1
      ans['answers'] = ''
    else:
      ans['answers'] = answer
      ans['paragraph_id'] = para_df[para_df['paragraph']==pred_para[best]]['id'].tolist()[0]
    pred_out.append(ans)

In [ ]:
# NOT allowed to make changes. 
from tqdm import tqdm
# All theme prediction.
questions = json.loads(pd.read_csv(ques_data_path).to_json(orient="records"))
theme_intervals = json.loads(pd.read_csv(theme_path).to_json(orient="records"))
pred_out = []
theme_inf_time = {}
execution_times = []
res=pd.DataFrame()
res['id']=None
res['A']=None
res['B']=None
res['C']=None
res['D']=None
res['E']=None
res.to_csv(f'/content/{m}_res.csv',index=False)
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions[int(theme_interval["start"]) - 1: int(theme_interval["end"])]
  theme = theme_ques[0]["theme"]
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  execution_times.append(execution_time)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv(result_path, index=False)
theme_inf_df = pd.DataFrame(list(theme_inf_time.items()),columns = ['theme','avg_inf_time']) 
theme_inf_df.to_csv(f'/content/{m}_inf_time.csv', index=False)

# Evaluation at k=3 (For Questions on DevRev Theme Only)


In [ ]:
 %cd /content/dhr

In [ ]:
# Allowed to make changes.
def get_theme_model(theme):
  global_model = nlp
  return global_model
def Average(lis):
  return sum(lis)/len(lis)

In [ ]:
sample_ground_truth_path = 'https://drive.google.com/uc?export=download&id=1--M9XlXEzOLDPCDT69LjmAsQYRPA2gZ2'
para_df=pd.read_csv(para_data_path)
ques_df=pd.read_csv(ques_data_path)
theme_df=pd.read_csv(theme_path)
ground_df=pd.read_csv(sample_ground_truth_path)
para_df=para_df.loc[para_df['theme']=='DevRev']
ques_df=ques_df.loc[ques_df['theme']=='DevRev']
theme_df=theme_df.loc[theme_df['theme']=='DevRev']
start = theme_df[theme_df['theme']=='DevRev']['start'].tolist()[0]
end = theme_df[theme_df['theme']=='DevRev']['end'].tolist()[0]
ground_df=ground_df.iloc[start-1:end]

In [ ]:
para_df2=pd.read_csv(f'{beir_data_path}/corpus.csv')

In [ ]:
# Allowed to make changes.
def pred_theme_ans(questions, theme_model, pred_out):
  theme = 'DevRev'
  if(synthetic_data==True):
    !python -m tevatron.datasets.beir.encode_and_retrieval_syn --theme f'{theme}' --model_name_or_path /content/drive/MyDrive/Assets/delade/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 
  elif(squad_dev_data==True):
    !python -m tevatron.datasets.beir.encode_and_retrieval_squad --theme f'{theme}' --model_name_or_path /content/drive/MyDrive/Assets/delade/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 
  else:
    !python -m tevatron.datasets.beir.encode_and_retrieval_new --theme f'{theme}' --model_name_or_path /content/dhr/DeLADE-CLS-P --m f'{m}_res.csv' --sim f'{similarity_method}' --data_path f'{beir_data_path}' 
  print(questions)
  start = time.time()
  for question in questions:
    print(question)
    ans = {}
    ans['question_id'] = question['id']    
    res5=pd.read_csv(f'/content/{m}_res.csv')
    l=len(para_df2.loc[para_df2['title']==theme].text.unique())
    
    if(l>=3):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].C.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]    
    elif(l==2):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].B.unique()[0]))].text.unique()[0],para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]   
    elif(l==1):
      pred_para=[para_df2.loc[para_df2['_id']==((res5.loc[res5['id']==question['id']].A.unique()[0]))].text.unique()[0]]
    scores = cross_model.predict([(question['question'],p) for p in pred_para])
    best = np.argmax(scores)
    result = theme_model(question=question['question'], context=pred_para[best])
    answer = result['answer']
    if result['score'] < threshold:
      ans['paragraph_id'] = -1
      ans['answers'] = ''
    else:
      ans['answers'] = answer
      ans['paragraph_id'] = para_df[para_df['paragraph']==pred_para[best]]['id'].tolist()[0]
    pred_out.append(ans)
  print ("time taken: " + str(time.time() - start) )

In [ ]:
# NOT allowed to make changes. 
from tqdm import tqdm
# All theme prediction.
questions = json.loads(ques_df.to_json(orient="records"))
theme_intervals = json.loads(theme_df.to_json(orient="records"))

pred_out = []
theme_inf_time = {}
execution_times = []
res=pd.DataFrame()
res['id']=None
res['A']=None
res['B']=None
res['C']=None
res['D']=None
res['E']=None
res.to_csv(f'/content/{m}_res.csv',index=False)
for theme_interval in tqdm(theme_intervals):
  theme_ques = questions
  theme = 'DevRev'
  # Load model fine-tuned for this theme.
  theme_model = get_theme_model(theme)
  execution_time = timeit.timeit(lambda: pred_theme_ans(theme_ques, theme_model, pred_out), number=1)
  execution_times.append(execution_time)
  theme_inf_time[theme_interval["theme"]] = execution_time * 1000 # in milliseconds.
  print (execution_time)
pred_df = pd.DataFrame.from_records(pred_out)
# Write prediction to a CSV file. Teams are required to submit this csv file.
pred_df.to_csv(result_path, index=False)
theme_inf_df = pd.DataFrame(list(theme_inf_time.items()),columns = ['theme','avg_inf_time']) 
theme_inf_df.to_csv(f'/content/{m}_inf_time.csv', index=False)